# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [1]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import glob
import cv2
import coco
import utils
import model as modellib
import visualize2 as visualize
import pandas as pd
from tesserocr import PyTessBaseAPI, PSM
import re
import datetime
import csv
import random

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
#IMAGE_DIR = os.path.join(ROOT_DIR, "images")

Using TensorFlow backend.


## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [2]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
  #  DETECTION_MIN_CONFIDENCE = 0.3

config = InferenceConfig()
config.display()


Configurations:
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [ 0.1  0.1  0.2  0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [ 123.7  116.8  103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           coco
NUM_CLASSES                    81
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000
POST_NMS_ROIS_TRAINI

## Create Model and Load Trained Weights

In [3]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [4]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

## Run Object Detection

In [8]:
def pedestrian_detection(time, image, display=True):
    print('------------------------------------------------------------')
    
    r = model.detect([image], verbose=1)[0]
    persons=r['class_ids']==1
#    if display:
#        cv2.imshow('',image)
#        k=cv2.waitKey(700)
#        cv2.destroyAllWindows()
    
    img = visualize.display_instances(image, r['rois'][persons], r['masks'][:,:,persons], 
                                          r['class_ids'][persons], class_names, r['scores'][persons])
    if sum(persons)<2:
        cv2.imwrite(directory+"\\"+str(time)+".jpg",img)
    
    if display:
        cv2.imshow('',img)
        k=cv2.waitKey(700)
        cv2.destroyAllWindows()
#добавила, чтобы ошибочные сохранить и посмотреть 

#    cv2.imwrite('D:\\_CV_PROJECT\\out2\\'+file[-10:],img)
        
    return sum(persons)

создание нового журнала

In [9]:
VIDEO_DIR = "Z:\\Converted\\1"

In [10]:
for num in [l[15:] for l in glob.glob(VIDEO_DIR+'\\*.mp4')]:
    #JOURNAL
    journal_columns=('TIME','number of persons')
    directory=VIDEO_DIR+'\\WARNINGS_'+num
    if not os.path.exists(directory):
        os.makedirs(directory)
    with open(directory+'\\journal.csv','w',newline='') as f:
                    writer=csv.DictWriter(f,journal_columns)
                    writer.writerow({'TIME':'time', 'number of persons':'number of persons'})
    journal=pd.DataFrame(columns=journal_columns)
    journal.to_csv(directory+'\\journal.csv')
    #END OF JOURNAL
    
    path=VIDEO_DIR+'\\'+num
    video = cv2.VideoCapture(path)
    i=0

    #while not (video.get(2)==1):
    while not int(video.get(1)==int(video.get(7))):
    #    print(video.get(2))
        img = video.read()[1]
        if (i%1000==0):
       # video.grab()
            print(num, video.get(1))
            TIME = int(video.get(0))
            data=pedestrian_detection(TIME,img,display=True)
            with open(directory+'\\journal.csv','a',newline='') as f:
                writer=csv.DictWriter(f,journal_columns)
                writer.writerow({'TIME':TIME, 'number of persons':data})

        i+=1

00012929_2900(1).mp4 1.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2900(1).mp4 1001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2900(1).mp4 2001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.700

00012929_2900(1).mp4 23001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2900(1).mp4 24001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2900(1).mp4 25001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -1

00012929_2900(1).mp4 45001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2900(1).mp4 46001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2900(1).mp4 47001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -1

00012929_2900(1).mp4 68001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2900(1).mp4 69001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2900(1).mp4 70001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -1

00012929_2900(1).mp4 91001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2900(1).mp4 92001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2900(1).mp4 93001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -1

...

00012929_2903.mp4 18001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2903.mp4 19001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2903.mp4 20001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000 

00012929_2903.mp4 41001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2903.mp4 42001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2903.mp4 43001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000 

00012929_2903.mp4 64001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2903.mp4 65001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2903.mp4 66001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000 

00012929_2903.mp4 87001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2903.mp4 88001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2903.mp4 89001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000 

00012929_2903.mp4 110001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2903.mp4 111001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000
image_metas              shape: (1, 89)               min:    0.00000  max: 1280.00000
00012929_2903.mp4 112001.0
------------------------------------------------------------
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000
molded_images            shape: (1, 1024, 1024, 3)    min: -123.700